Installation. In Julia REPL this can be done easier using the packaging mode
(typing `]add COBREXA`)

In [ ]:
import Pkg
Pkg.add("COBREXA")

using COBREXA

Let's first get a simple model to have a look at

In [ ]:
import Downloads
Downloads.download(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
)

Load the model

In [ ]:
ecoli = load_model("e_coli_core.json");

have a look at what the model contains. First, metabolites

In [ ]:
metabolites(ecoli)

reactions

In [ ]:
reactions(ecoli)

we can have a look at what each reaction does

In [ ]:
reaction_stoichiometry(ecoli, "PFK")

or look at which metabolites belong to which compartments (Julia looping!)

In [ ]:
[m => metabolite_compartment(ecoli, m) for m in metabolites(ecoli)]

or try to get human-readable names from the reactions

In [ ]:
sort([r => reaction_name(ecoli, r) for r in reactions(ecoli)])

this is useful to e.g. quickly look up dehumanized gene IDs

In [ ]:
gs = genes(ecoli)
Dict(gene_name.(Ref(ecoli), gs) .=> gs)

the model's internals are described by a bipartite graph between reactions
and metabolites, commonly stoichiometry

In [ ]:
stoichiometry(ecoli)

(the matrix is likely zoomed out, but otherwise it's a normal matrix)

Let's try to build a tiny custom model
ref: https://lcsb-biocore.github.io/COBREXA.jl/stable/examples/04b_standardmodel_construction/

The model type for "manually constructed" models we call a StandardModel
because it is kinda standard way to handle stuff in cobra community.
Technically we can convert the above model to StandardModel and play with it:

In [ ]:
ecoli = convert(StandardModel, ecoli)
ecoli.reactions["PFK"]
ecoli.reactions["PFK"].lb = 0.5

But we want to make a completely custom model

In [ ]:
model = StandardModel("MyModel")

let's make some metabolites

In [ ]:
a = Metabolite("a", name = "molecule A", formula = "H2O", compartment = "outside")
b = Metabolite("b") #details omitted for demonstration
c = Metabolite("c")

Push the prepared metabolites into the model (the ! stands for "execute!", it
is a syntactic convention for warning that the function changes some of the
parameters (the model) in place)

In [ ]:
add_metabolites!(model, [a, b, c])

let's make some reactions

In [ ]:
b1 = Reaction("b1", lb = 0.0, ub = 10.0);
b1.metabolites = Dict("a" => 1);

b2 = Reaction("b2", lb = 0.0, ub = 10.0);
b2.metabolites = Dict("b" => -1);

b3 = Reaction("b3", lb = 0.0, ub = 10.0);
b3.metabolites = Dict("c" => -1);

add the reactions to the model

In [ ]:
add_reactions!(model, [b1, b2, b3])

shortcut for the above, makes an array of 3 reactions at once

In [ ]:
list_of_reactions = [Reaction("v$i", lb = 0.0, ub = 10.0) for i = 1:3]

...and we can fill them in

In [ ]:
list_of_reactions[1].metabolites = Dict("a" => -1, "b" => 1)
list_of_reactions[2].metabolites = Dict("a" => -1, "c" => 1)
list_of_reactions[3].metabolites = Dict("a" => 1, "c" => -1)

# INspecting one of the reactions
rxn = list_of_reactions[1]
rxn

add the reactions to the model

In [ ]:
add_reactions!(model, list_of_reactions)

let's have a look at what it looks like as a matrix

In [ ]:
stoichiometry(model)

Now let's try to numerically solve the model. First, we need a linear solver;
GLPK will do a good job for this purpose. Other alternatives include Tulip
(native interior-point solver), OSQP and Clarabel (for quadratic problems),
SCIP (free and fast), Clp, and very good commercial solvers include Gurobi
and CPLEX (you will need to install the licenses for these manually).

In [ ]:
Pkg.add("GLPK")
using GLPK

# Pkg.add("SCIP")
# using SCIP

"Solving" the model is finding optimum with respect to some reaction, thus we
first need to choose the objective that the solver should actually optimize.
Here, let's maximize the flux through the reaction "v1".

In [ ]:
change_objective!(model, "v1")

You can observe the result in the model reactions' objective coefficients
(and fine-tune that manually if needed):

In [ ]:
model.reactions["v1"].objective_coefficient

Flux balance analysis finds a steady state flux through the model which is
within the reaction flux lower and upper bounds:

In [ ]:
solution = flux_balance_analysis(model, GLPK.Optimizer)

The above returned a solved optimization model description. From that we can
extract the actual value of the objective function:

In [ ]:
solved_objective_value(solution)

...and the complete description of the flux through the reactions

In [ ]:
flux_vector(model, solution)

...or a little better as a dictionary

In [ ]:
sol_dict = flux_dict(model, solution)

In [ ]:
Pkg.add(["DataFrames", "CSV"])
using DataFrames, CSV

In [ ]:
df = DataFrame(reaction = collect(keys(sol_dict)), flux = collect(values(sol_dict)))
df

typically, one does all of this in one step with a shortcut function:

In [ ]:
flux = flux_balance_analysis_dict(model, GLPK.Optimizer)

As a check, we can verify that the amount of metabolites in the model indeed
stays balanced; if the solver worked, this vector should be zero (or within
the numerical tolerance of zero):

In [ ]:
stoichiometry(model) * flux_vector(model, solution)

In [ ]:
Pkg.add("SCIP")
using SCIP

In [ ]:
flux = flux_balance_analysis_dict(model, SCIP.Optimizer)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*